# Importing library

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import cleaner
import hazm
from collections import defaultdict




## Load data

* In this block we load csv file and stop words

In [2]:
data = pd.read_csv('./culture.csv')
with open('Verb.txt') as f:
    verb = f.readlines()
verb = [unicode(i.strip(),'utf-8') for i in verb]

In [195]:
def is_verb(word):
    lemmet = hazm.Lemmatizer()
    if (('#' in lemmet.lemmatize(word)) or (word in verb)):
        return True
    return False

## Delete stop words and punctuation marks

In [3]:
def delete_stop_words(word):    
    if (word in verb):
        return False
    else:
        n = re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)
    return n

## Cleaning the thex

- This function tokenize the text's word and use delete_stop_words for each word to delete or keep the word in a list 
- Then return a list contain word and postion of each one

In [20]:
def clean_text(text):
    normalizer = hazm.Normalizer()
    text = normalizer.affix_spacing(text)
    sentence = nltk.word_tokenize(unicode(text,'utf-8'))
    clean_words = [] 
    for i in range(len(sentence)):
        res = delete_stop_words(sentence[i])
        if res:
            clean_words.append({'Position':i,'Word':res})
    return clean_words

## Take a sample
- Take a sample from all data and clean it . "words" variable contain the all words we need

In [21]:
text = data['mText'][6]
all_words = nltk.word_tokenize(unicode(text,'utf-8'))
words = clean_text(text)

In [6]:
lemmet = hazm.Lemmatizer()
for i in words:
    i['Word'] = lemmet.lemmatize(i['Word'])

In [22]:
position_dict= defaultdict(list)
for i in words:
    position_dict[i['Word']].append(i['Position'])
    
for i in position_dict:
    position_dict[i] = sorted(position_dict[i])

In [23]:
life_time = defaultdict(int)
for i in position_dict:
    n = position_dict[i][-1]-position_dict[i][0] -1 
    if n == -1 :
        n = 0
    life_time[i] = float(n)/len(all_words)

In [24]:
words_dict = defaultdict()
for i in position_dict:
    n = defaultdict()
#     n['Word'] = i
    n['Position'] = position_dict[i]
    n['life_time'] = life_time[i]
    n['tf'] = len(position_dict[i])
    words_dict[i] = n

In [25]:
mean = float(sum([words_dict[i]['tf'] for i in words_dict]))/len(words_dict)
for word in words_dict:

    words_dict[word]['Var'] = words_dict[word]['tf'] - mean

In [26]:
alpha = 0.55
for word in words_dict:
    words_dict[word]['Weight'] = (((1-alpha)*words_dict[word]['life_time']) + alpha*(words_dict[word]['tf']+words_dict[word]['Var'])) - words_dict[word]['Var']

    

In [27]:
sorted_list = sorted(words_dict,key = lambda x : words_dict[x]['Weight'],reverse=True)

In [28]:
w = nltk.word_tokenize(unicode(text,'utf-8'))

In [30]:
for i in sorted_list:
    print(i)
    print(words_dict[i])

هنر
defaultdict(None, {'tf': 12, 'Position': [12, 29, 52, 69, 79, 120, 132, 136, 142, 239, 247, 319], 'life_time': 0.6891891891891891, 'Weight': 2.2288401711063592, 'Var': 10.402877697841726})
جشنواره
defaultdict(None, {'tf': 10, 'Position': [11, 22, 35, 46, 178, 210, 230, 246, 259, 404], 'life_time': 0.8828828828828829, 'Weight': 2.1160023332685203, 'Var': 8.402877697841726})
داور
defaultdict(None, {'tf': 7, 'Position': [57, 91, 149, 255, 278, 352, 362], 'life_time': 0.6846846846846847, 'Weight': 1.7268131440793324, 'Var': 5.402877697841727})
نقاش
defaultdict(None, {'tf': 6, 'Position': [19, 75, 156, 174, 244, 257], 'life_time': 0.5337837837837838, 'Weight': 1.558907738673926, 'Var': 4.402877697841727})
تجسم
defaultdict(None, {'tf': 6, 'Position': [13, 30, 47, 53, 179, 248], 'life_time': 0.527027027027027, 'Weight': 1.5558671981333863, 'Var': 4.402877697841727})
آثار
defaultdict(None, {'tf': 7, 'Position': [253, 256, 279, 318, 363, 371, 380], 'life_time': 0.28378378378378377, 'Weight'

In [199]:
related_word = defaultdict(list)
for word in sorted_list[:30]:
    l_pos = 0 
    for pos in words_dict[word]['Position']:
        if pos-5 < l_pos+5:
            pos = l_pos - pos + 10
        l_pos = pos
        related_word[word].append(w[pos-5:pos+5])

In [201]:
for i in related_word:
    t = []
    for j in related_word[i]:
        t+= j
    t = [k for k in t if len(k)>=3 or not is_verb(k)]

    related_word[i] = [lemmet.lemmatize(re.sub(u'[\u060c|.|:|;|!|@|#|$|%|^|&|*|\xab|\xbb|/]','',word)).strip() for word in t ]

In [204]:
n = defaultdict()
for i in related_word:
    m = pd.Series(related_word[i]).value_counts()
    n[i] = dict(m)
    


In [208]:
probability = defaultdict(dict)
for i in n:
    for j in n[i]:
        if(len(j)<3):
            continue
        
        try:
            
            prob = (float(n[i][j])/words_dict[j]['tf'])
            if prob>1 :
                prob = 0
            probability[i][j] = prob

        except:
            probability[i][j] = 0
        
        


************
باعث
************
وهله
************
هنر
************
جشنواره
************
شعار
************
وجود
************
دبیر
************
رشته
************
یازدهمین
************
انتخاب
************
جهانی
************
نقاش
************
دوره
************
موفق
************
آثار
************
جدید
************
تجسم
************
دولت
************
داور
************
هنرمند
************
حتما
************
جدا
************
اساس
************
توجه
************
خوب
************
مراحل
************
شاخه
************
را
************
هویت
************
فجر


In [209]:
for i in probability:
    sort = sorted(probability[i].items(),key=lambda x : x[1],reverse=True)
    probability[i] = sort


In [210]:
for i in probability:
    print(i)
    for j,x in probability[i]:
        print(j)
        print(x)
    print('*************')

باعث
تصمیم
1.0
جداگانه
1.0
باعث
1.0
خلط
1.0
مبحث
1.0
انجام
1.0
گمان
1.0
ارزیابی‌ها
1.0
بهتر
1.0
صورت
1.0
جوان
1.0
درست
1.0
رشته
0.2
داور
0.142857142857
هنر
0.0833333333333
کنار
0
یکدیگر
0
شد#شو
0
بسیار
0
کرد#کن
0
#است
0
*************
وهله
حالت
1.0
پیدا
1.0
تصاویر
1.0
وهله
1.0
اساس
0.5
آثار
0.142857142857
داور
0.142857142857
خود
0
اول
0
باید
0
کند
0
برای
0
*************
هنر
ده‌گانه
1.0
گزارش
1.0
حوزه
1.0
آخرین
1.0
دستاورد
1.0
خبرگزاری
1.0
وکیل
1.0
جهان
1.0
صورت
1.0
قرار
1.0
تفکیک
1.0
اجتماع
1.0
جداگانه
1.0
شد#شو
1.0
مربوط
1.0
جدید
1.0
بزرگ
1.0
ارزش‌گذاری
1.0
اشاره
1.0
فارس
1.0
احمد
1.0
تدوین
1.0
شرایط
1.0
نقل
1.0
گمان
1.0
امور
1.0
هنر
0.916666666667
یازدهمین
0.666666666667
تجسم
0.666666666667
رشته
0.6
دبیر
0.5
دوره
0.5
هویت
0.5
حتما
0.5
توجه
0.5
شاخه
0.5
باعث
0.333333333333
وجود
0.25
فجر
0.25
نقاش
0.166666666667
آثار
0.142857142857
جشنواره
0.1
کنار
0
رساند#رسان
0
یکدیگر
0
ستاد
0
گرفت#گیر
0
چندان
0
شده
0
#است
0
داشت
0
توانست#توان
0
سپرد#سپر
0
باید
0
اطلاع
0
کرد#کن
0
این
0
دیگر
0
دنبال
0
